In [1]:
import torch
import numpy as np
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
from torch_optimizer import Lookahead


from model import KeywordSpottingModel_with_cls
from data_loader import load_speech_commands_dataset, TFDatasetAdapter, load_bg_noise_dataset
from utils import set_memory_GB,print_model_size, log_to_file, EarlyStopping
from augmentations import add_time_shift_and_align, add_silence
from train_utils import trainig_loop




/usr/local/lib/python3.10/dist-packages/mamba_ssm/ops/selective_scan_interface.py:164: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, xz, conv1d_weight, conv1d_bias, x_proj_weight, delta_proj_weight,
/usr/local/lib/python3.10/dist-packages/mamba_ssm/ops/selective_scan_interface.py:240: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, dout):
/usr/local/lib/python3.10/dist-packages/mamba_ssm/ops/triton/layer_norm.py:986: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(
/usr/local/lib/python3.10/dist-packages/mamba_ssm/ops/triton/layer_norm.py:1045: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type=

In [2]:
torch.cuda.is_available()

True

In [3]:
set_memory_GB(1)

Memory fraction set to 0.022458079576498518
Memory fraction in GB: 1.0


In [4]:
train_ds, val_ds, test_ds, silence_ds , info = load_speech_commands_dataset()
bg_noise_ds = load_bg_noise_dataset()

2024-09-11 14:45:22.995166: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-11 14:45:22.998780: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-11 14:45:23.006330: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-11 14:45:23.018475: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-11 14:45:23.021846: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attemptin

In [5]:
print(train_ds)

<_FilterDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.int16, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>


In [6]:
# maintain seed for repructablity
np.seed = 42
# tf.random.set_seed(42)
torch.manual_seed(0)

In [7]:
label_names = ['down', 'go', 'left', 'no', 'off', 'on', 'right', 'stop', 'up', 'yes']
print(label_names)

['down', 'go', 'left', 'no', 'off', 'on', 'right', 'stop', 'up', 'yes']


In [8]:
# augmentations = [
#     lambda x: add_time_shift_and_align(x),
# ]

In [9]:
# # Convert the TFDS dataset to a PyTorch Dataset
# fixed_length = 16000
# n_mfcc = 13
# n_fft = 400
# hop_length = 160
# n_mels = 40
# pytorch_train_dataset = TFDatasetAdapter(train_ds, fixed_length, n_mfcc, n_fft, hop_length, n_mels, augmentations)
# pytorch_val_dataset = TFDatasetAdapter(val_ds, fixed_length, n_mfcc, n_fft, hop_length, n_mels, augmentations=None)

In [10]:
# # Create a DataLoader to feed the data into the model
# batch_size = 32
# train_loader = DataLoader(pytorch_train_dataset, batch_size=batch_size, shuffle=True,num_workers=4,prefetch_factor=2)
# val_loader = DataLoader(pytorch_val_dataset, batch_size=batch_size, shuffle=False,num_workers=4,prefetch_factor=2)

In [11]:
# for audio, label in train_loader:
#     print(audio.shape, label.shape)
#     break


In [12]:
# # Varify Tensor's shape
# # Example audio sample
# from librosa.feature import mfcc
# audio = np.random.randn(16000).astype(np.float32)  # Simulate 1-second audio at 16kHz

# # Compute MFCC features
# mfcc_features = mfcc(y=audio, sr=16000, n_mfcc=13, n_fft=400, hop_length=160, n_mels=40, fmin=0, fmax=8000)

# print(f'MFCC shape: {mfcc_features.shape}')  # Expected: (13, num_frames)

# Training loop

# With L2 regulariztion AND Droput layer

In [21]:
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import tqdm as tqdm

def objective(trial):
    # maintain seed for repructablity
    np.seed = 42
    # tf.random.set_seed(42)
    torch.manual_seed(0)

    # Suggest hyperparameters
    # input_dim = trial.suggest_int('input_dim', 12, 20)  # MFCC features, likely between 13-40
    # d_model = trial.suggest_int('d_model', 2, 128)  # Dimension of model layers
    d_state = trial.suggest_int('d_state', 8, 64)  # Dimension of hidden state
    d_conv = trial.suggest_int('d_conv', 2, 16)  # Dimension of convolutional layers
    expand = trial.suggest_int('expand', 2, 4)  # Expansion factor for Mamba layer
    batch_size = trial.suggest_int('batch_size', 8, 64)  # Batch size
    dropout_rate = trial.suggest_float('dropout_rate', 0.05, 0.7)  # Dropout rate
    num_mamba_layers = trial.suggest_int('num_mamba_layers', 1, 2)  # Number of Mamba layers
    n_mfcc = trial.suggest_int('n_mfcc', 6, 26)  # Number of MFCC features
    n_fft = trial.suggest_int('n_fft', 200, 800)  # FFT window size
    hop_length = trial.suggest_int('hop_length', 80, 640)  # Hop length
    n_mels = trial.suggest_int('n_mels', 20, 100)  # Number of Mel bins
    noise_level = trial.suggest_float('noise_level', 0.0, 0.8)  # Noise level for augmentation
    augmentations = [
    lambda x: add_time_shift_and_align(x,trial.suggest_int('shift', 0, 150)),
    ]
    
    # Convert the TFDS dataset to a PyTorch Dataset
    fixed_length = 16000
    pytorch_train_dataset = TFDatasetAdapter(train_ds, bg_noise_ds, fixed_length, n_mfcc, n_fft, hop_length, n_mels, augmentation = augmentations,noise_level=noise_level)
    pytorch_val_dataset = TFDatasetAdapter(val_ds, None, fixed_length, n_mfcc, n_fft, hop_length, n_mels, augmentation=None)

    # Define DataLoader
    train_loader = DataLoader(pytorch_train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, prefetch_factor=2)
    val_loader = DataLoader(pytorch_val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, prefetch_factor=2)


    # Init early stopping
    early_stopping = EarlyStopping(patience=5, min_delta=0.00001)
    

    # Initialize the model
    model = KeywordSpottingModel_with_cls(
        input_dim=n_mfcc*3, 
        d_model= (16000 // hop_length) + 1 + 1  ,
        d_state=d_state, 
        d_conv=d_conv, 
        expand=expand,
        dropout_rate=dropout_rate,
        label_names=label_names,
        num_mamba_layers=num_mamba_layers
    ).to("cuda")

    # Define optimizer
    lr = trial.suggest_loguniform('lr', 1e-4, 1e-2)
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-4)
    base_optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    optimizer = Lookahead(base_optimizer, k=5, alpha=0.5)  # Wrap around Adam optimizer with Lookahead
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)    
    # Define loss function
    criterion = nn.CrossEntropyLoss().to("cuda")  # Loss function with class weights

    log_to_file(f"\n\nStarting training with hyperparameters: {trial.params}","optuna.log\n\n")
    # Training loop
    train_loss = 0.0
    correct_train = 0
    total_train = 0

    num_epochs = 100  # Set to higher value depending on compute resources
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        
        for audio, labels in train_loader:
            audio, labels = audio.to("cuda"), labels.to("cuda")
            optimizer.zero_grad()
            outputs = model(audio)
            loss = criterion(outputs, labels)
            loss.backward()
            train_loss += loss.item()
            optimizer.step()
            running_loss += loss.item()
            # Calculate training accuracy
            _, predicted = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_accuracy = 100 * correct_train / total_train
        log_to_file(f"Epoch {epoch} - Training Loss: {train_loss}, Training Accuracy: {train_accuracy}","optuna.log")

        val_loss = 0.0
        correct_val = 0
        total_val = 0

        model.eval()
        with torch.no_grad():
            for audio, labels in val_loader:
                audio, labels = audio.to("cuda"), labels.to("cuda")
                outputs = model(audio)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
                _, predicted = torch.max(outputs, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()

        val_loss_avg = val_loss / len(val_loader)
        val_accuracy = 100 * correct_val / total_val
        log_to_file(f"Epoch {epoch} - Validation Loss: {val_loss_avg}, Validation Accuracy: {val_accuracy}","optuna.log")
        
        # Step the scheduler based on validation loss
        scheduler.step(val_loss_avg)

        # Early stopping or other criteria could be added here
        trial.report(val_loss_avg, epoch)
        if early_stopping.step(val_loss_avg):
            break
        
        # If the trial is pruned (i.e., the result is worse than expected), stop early
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
        
        # load test data
        pytorch_test_dataset = TFDatasetAdapter(test_ds,None, fixed_length, n_mfcc, n_fft, hop_length, n_mels, augmentation=None)
        test_loader = DataLoader(pytorch_test_dataset, batch_size=batch_size, shuffle=False,num_workers=4,prefetch_factor=2)

        # Evaluate the model on the test set
        accuracy = 0
        total = 0
        model.eval()

        with torch.no_grad():
            for audio, labels in test_loader:
                audio, labels = audio.to("cuda"), labels.to("cuda")
                outputs = model(audio)
                _, predicted = torch.max(outputs, 1)
                
                total += labels.size(0)
                accuracy += (predicted == labels).sum().item()
        test_accuracy = 100 * accuracy / total
        log_to_file(f"Epoch {epoch} - Test Accuracy: {test_accuracy}","optuna.log")
    
    # Return the final validation loss as the metric to minimize
    log_to_file(f"Final Validation Loss: {val_loss_avg}, Test Accuracy: {test_accuracy}","optuna.log","\n\n")
    
    return val_loss_avg


In [20]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100,show_progress_bar=True)

[I 2024-09-12 12:48:11,020] A new study created in memory with name: no-name-43738d22-6561-4398-aba3-e26b171407da


  0%|          | 0/100 [00:00<?, ?it/s]

/tmp/ipykernel_204242/3165251575.py:59: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-4, 1e-2)
/tmp/ipykernel_204242/3165251575.py:60: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-4)


In [ ]:
# save the study
import joblib
joblib.dump(study, 'study.pkl')

['study.pkl']